# Search Skill Validation

## Setup

1. Run the Search Skill server either as Docker container or locally. Just make sure it serves at: `http://127.0.0.1:6002`
2. Place images under `cognitiveSearch\playground\blob_images` following the same directory structure as in storage. Example: `869158001/0869158001.jpg``
3. Run the notebook

The notebook is using as baseline the same search queries used for the initial search evaluation. Better result will be obtained when using the real "recommender skill"

In [1]:
%pip install ipywidgets
%pip install jupyter_contrib_nbextensions


  Using cached ipywidgets-8.1.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.9-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.1-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.9-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.9-py3-none-any.whl (2.3 MB)
  Using cached jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
  Using cached jupyter_contrib_core-0.4.2-py2.py3-none-any.whl
  Using cached jupyter_highlight_selected_word-0.2.0-py2.py3-none-any.whl (11 kB)
  Using cached jupyter_nbextensions_configurator-0.6.3-py2.py3-none-any.whl (466 kB)
  Using cached notebook-7.0.6-py3-none-any.whl.metadata (10 kB)
  Using cached lxml-4.9.3-cp310-cp310-win_amd64.whl.metadata (3.9 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached bleach-6.1.0-py3-n

In [2]:
import pandas as pd
from IPython.display import display, HTML
from IPython.display import Image

In [3]:
def merge_dfs(dfs: list):
    merged = None
    for df in dfs:
        if merged is None:
            merged = df
            continue

        merged = pd.merge(merged, df, on="id", how="inner")
    return merged


def display_image(path):
    return f'<img src="{path}" width="100" />'

def enable_col_show_image(df, source_col: str, output_col: str = None):
    if output_col is None:
        output_col = source_col
    df[source_col] = df[output_col].apply(display_image)

def create_output_results_df(output_list: list, additional_cols: list = []):
    display_cols = ["id", "article_id", "score", "image_path", "local_filepath"]
    if additional_cols:
        display_cols += additional_cols

    df = pd.DataFrame(output_list, columns=display_cols)
    enable_col_show_image(df, "image_html", "local_filepath")
    df = df.drop(columns=["local_filepath", "image_path"], axis=1)

    return df

def display_df_with_images(df):
    display(HTML(df.to_html(escape=False)))

In [5]:
cached_gptv_queries_df = pd.read_csv("./csv/dual_input_gptv_input.csv", encoding="utf-8", keep_default_na=False)
cached_gptv_queries_df = cached_gptv_queries_df.set_index("id")
cached_gptv_queries_df = cached_gptv_queries_df[["gptv-input"]]
cached_gptv_queries_df = cached_gptv_queries_df.rename(index=str, columns={f"gptv-input": "gptv-description"})

# removing repeated entries
cached_gptv_queries_df = cached_gptv_queries_df[cached_gptv_queries_df.index.str.contains("-1")]
cached_gptv_queries_df.index = cached_gptv_queries_df.index.str.replace("-1", "")

display(cached_gptv_queries_df)

# function to be used as a substitute of "generate_gptv_prompt" - using same interface
def get_cached_gptv_image_description(image_path: str = "", text_input: str = ""):
    return cached_gptv_queries_df.loc[text_input]["gptv-description"]

,gptv-description
id,
I’m looking for a jacket like this one except darker,I’m looking for a jacket like this one except ...
How about a green one instead,How about a green one instead: This is a vibra...
Do you have a sleeper like this with hearts,Do you have a sleeper like this with hearts: T...
Shoes to go with,Shoes to go with: This product is a stylish an...
"something more casual, like this but mesh and thinner sole","something more casual, like this but mesh and ..."
"I’d like a polo t-shirt to go with my khakis, ignore the shirts","I’d like a polo t-shirt to go with my khakis, ..."
"I’d like a green polo t-shirt to go with my khakis, ignore the shirts.",I’d like a green polo t-shirt to go with my kh...
I’m looking for some baby slippers,I’m looking for some baby slippers: These are ...
"I’m looking for some baby slippers, something more Halloween, like this but a slipper for a 6 month old","I’m looking for some baby slippers, something ..."


In [15]:
import requests

def search_skill_search(search_query: str, results: int = 3):
    url = "http://127.0.0.1:3001/imageSearch"
    headers = {'Content-Type': 'application/json'}
    body = {"search_queries": [
        {
        "search_query": search_query,
        "max_results_count": results
        }
    ]}
    response = requests.post(url, json=body, headers=headers)
    return response.json()

def extract_search_result_props(result: dict):
    collection = list()
    data: dict = result.get("results")[0]
    query = data.get("search_query")
    results = data.get("search_results")

    for i, item in enumerate(results):
        result_id = f"{query}-{i}"
        filepath = item.get("image_filepath")
        collection.append({
            "id": result_id,
            "article_id": item.get("article_id"),
            "product_name": item.get("prod_name"),
            "local_filepath": f"./blob_images/{filepath}"
        })

    return collection


In [20]:
query = "blue pants"
results = search_skill_search(query)
output = extract_search_result_props(results)
output

[{'id': 'blue pants-0',
  'article_id': '869158001',
  'product_name': 'DRAKE sweatpants TP',
  'local_filepath': './blob_images/869158001/0869158001.jpg'},
 {'id': 'blue pants-1',
  'article_id': '800110001',
  'product_name': 'Newton skinny trs',
  'local_filepath': './blob_images/800110001/0800110001.jpg'},
 {'id': 'blue pants-2',
  'article_id': '525838008',
  'product_name': 'TUCK CLASSIC SWEATPANTS',
  'local_filepath': './blob_images/525838008/0525838008.jpg'}]

In [23]:
df = pd.DataFrame(output)
df

,id,article_id,product_name,local_filepath
0,blue pants-0,869158001,DRAKE sweatpants TP,./blob_images/869158001/0869158001.jpg
1,blue pants-1,800110001,Newton skinny trs,./blob_images/800110001/0800110001.jpg
2,blue pants-2,525838008,TUCK CLASSIC SWEATPANTS,./blob_images/525838008/0525838008.jpg


In [26]:
results = list()
for index, row in cached_gptv_queries_df.iterrows():
    search_query = row["gptv-description"]
    search_results = search_skill_search(search_query)
    output = extract_search_result_props(search_results)
    results += output

In [30]:
df = pd.DataFrame(results)

In [31]:
df["local_filepath"] = df["local_filepath"].apply(display_image)
display_df_with_images(df)

,id,article_id,product_name,local_filepath
0,"I’m looking for a jacket like this one except darker: This is a vibrant cobalt blue puffer jacket designed to provide warmth and comfort in cold weather. The jacket features a horizontal quilted pattern that ensures even distribution of insulation throughout the garment, promoting consistent warmth. It includes a full-length zipper, making it easy to put on and take off and allowing the wearer to adjust the level of ventilation as needed.\n\nThe jacket is equipped with a generously sized hood that is meant to offer additional protection against the elements. The hood is also quilted and looks to be non-detachable. The elasticized cuffs and hem are notable features that help in retaining heat, preventing cold air from entering, and ensuring a snug fit.\n\nConstructed from what appears to be a glossy, water-resistant material, this jacket is also likely to provide some level of protection against light rain and snow, making it a versatile piece for various weather conditions. \n\nIt is suitable for daily wear during the colder seasons, outdoor activities, and can be paired with different casual attire for a stylish yet functional cold-weather ensemble. The bold blue color is eye-catching and adds a pop of color to the gloomy winter landscape.-0",680141001,SB Mason insulation,
1,"I’m looking for a jacket like this one except darker: This is a vibrant cobalt blue puffer jacket designed to provide warmth and comfort in cold weather. The jacket features a horizontal quilted pattern that ensures even distribution of insulation throughout the garment, promoting consistent warmth. It includes a full-length zipper, making it easy to put on and take off and allowing the wearer to adjust the level of ventilation as needed.\n\nThe jacket is equipped with a generously sized hood that is meant to offer additional protection against the elements. The hood is also quilted and looks to be non-detachable. The elasticized cuffs and hem are notable features that help in retaining heat, preventing cold air from entering, and ensuring a snug fit.\n\nConstructed from what appears to be a glossy, water-resistant material, this jacket is also likely to provide some level of protection against light rain and snow, making it a versatile piece for various weather conditions. \n\nIt is suitable for daily wear during the colder seasons, outdoor activities, and can be paired with different casual attire for a stylish yet functional cold-weather ensemble. The bold blue color is eye-catching and adds a pop of color to the gloomy winter landscape.-1",393447015,Leia Parka,
2,"I’m looking for a jacket like this one except darker: This is a vibrant cobalt blue puffer jacket designed to provide warmth and comfort in cold weather. The jacket features a horizontal quilted pattern that ensures even distribution of insulation throughout the garment, promoting consistent warmth. It includes a full-length zipper, making it easy to put on and take off and allowing the wearer to adjust the level of ventilation as needed.\n\nThe jacket is equipped with a generously sized hood that is meant to offer additional protection against the elements. The hood is also quilted and looks to be non-detachable. The elasticized cuffs and hem are notable features that help in retaining heat, preventing cold air from entering, and ensuring a snug fit.\n\nConstructed from what appears to be a glossy, water-resistant material, this jacket is also likely to provide some level of protection against light rain and snow, making it a versatile piece for various weather conditions. \n\nIt is suitable for daily wear during the colder seasons, outdoor activities, and can be paired with different casual attire for a stylish yet functional cold-weather ensemble. The bold blue color is eye-catching and adds a pop of color to the gloomy winter landscape.-2",393447009,Leia Parka,
3,"How about a green one instead: This is a vibrant blue quilted down jacket, des